## PS 7 Chen Anhua



In [1]:
import pandas as pd
import numpy as np
import sklearn
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split
from pylab import rcParams
import warnings
import matplotlib.pyplot as plt
import seaborn as sb
from sklearn.cross_validation import train_test_split
from sklearn import metrics 
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import LeaveOneOut, KFold
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier

C:\Users\Lenovo\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
df = pd.read_csv("auto.csv", na_values = "?")
mpg_median = df["mpg"].median()
df["mpg_high"] = df["mpg"].apply(lambda x: 1 if x >= mpg_median else 0)
df["mpg_high"] = df["mpg_high"].astype('category')
df["const"] = 1

In [3]:
## test the cleaned data
df.dropna(inplace=True)
df.isnull().sum()

mpg             0
cylinders       0
displacement    0
horsepower      0
weight          0
acceleration    0
year            0
origin          0
name            0
mpg_high        0
const           0
dtype: int64

In [4]:
df["horsepower"] = df["horsepower"].astype("int")
Xvars = df[["cylinders", "displacement", "horsepower", "weight", "acceleration", "year", "origin"]].values
yvals = df[["mpg_high"]].values

#### (a) Logistic regression

In [5]:
k = 4
kf = KFold(n_splits = k, shuffle = True, random_state = 15)
kf.get_n_splits(Xvars)

MSE_vec_kf = np.zeros(k)
err1_rate_vec = np.zeros(k)
err0_rate_vec = np.zeros(k)

k_ind = int(0)
for train_index, test_index in kf.split(Xvars):
    
    X_train, X_test = Xvars[train_index], Xvars[test_index]
    y_train, y_test = yvals[train_index], yvals[test_index]
    
    LogReg = LogisticRegression()
    LogReg.fit(X_train, y_train)
    y_pred = LogReg.predict(X_test)
    y_test = y_test.flatten()
    
    #logical filter
    a = (y_test != y_pred)
    b0 = y_pred == 0
    b1 = y_pred == 1
    err0_rate = (b0 & a).sum() / b0.sum() 
    err1_rate = (b1 & a).sum() / b1.sum() 
    err0_rate_vec[k_ind] = err0_rate
    err1_rate_vec[k_ind] = err1_rate
    MSE_vec_kf[k_ind] = (y_test != y_pred).mean()
    
    print('When k =', k_ind)
    print(classification_report(y_test, y_pred))
    print('error rate for category 0 is', err0_rate)
    print('error rate for category 1 is', err1_rate)
    
    
    k_ind += 1

print("============ summary ===============")
print('average MSE k-fold for both categories', MSE_vec_kf.mean())
print('average MSE standard err for both categories', MSE_vec_kf.std())
print('average MSE k-fold for category 0', err0_rate_vec.mean())
print('average MSE standard err for category 0', err0_rate_vec.std())
print('average MSE k-fold for category1', err1_rate_vec.mean())
print('average MSE standard err for category1', err1_rate_vec.std())

When k = 0
             precision    recall  f1-score   support

          0       0.94      0.89      0.92        55
          1       0.87      0.93      0.90        43

avg / total       0.91      0.91      0.91        98

error rate for category 0 is 0.0576923076923
error rate for category 1 is 0.130434782609
When k = 1
             precision    recall  f1-score   support

          0       0.88      0.91      0.90        47
          1       0.92      0.88      0.90        51

avg / total       0.90      0.90      0.90        98

error rate for category 0 is 0.122448979592
error rate for category 1 is 0.0816326530612
When k = 2
             precision    recall  f1-score   support

          0       0.85      0.87      0.86        45
          1       0.88      0.87      0.88        53

avg / total       0.87      0.87      0.87        98

error rate for category 0 is 0.152173913043
error rate for category 1 is 0.115384615385
When k = 3
             precision    recall  f1-score   

C:\Users\Lenovo\Anaconda3\lib\site-packages\sklearn\utils\validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


#### (b) RF

In [6]:
Random_forest = RandomForestClassifier(n_estimators = 20, max_features=2, bootstrap=True,
                                       oob_score=True, random_state=25)
Random_forest.fit(Xvars, yvals)

C:\Users\Lenovo\Anaconda3\lib\site-packages\ipykernel\__main__.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  app.launch_new_instance()


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features=2, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=20, n_jobs=1, oob_score=True, random_state=25,
            verbose=0, warm_start=False)

In [7]:
## transform continuous to categorical
y_pred = Random_forest.oob_decision_function_.T[1]
pred = []    
for y in y_pred:
    if y >= 0.5:
        pred.append(1)
    else:
        pred.append(0)
pred = np.array(pred)

MSE = mean_squared_error(pred, yvals)
        
print("The MSE for the RF is", MSE)

pred = pred.tolist()
pred_0 = []
yval_0 = []
pred_1 = []
yval_1 = []
for i, p in enumerate(pred):
    if p < 0.5:
        pred_0.append(0)
        yval_0.append(yvals[i])
    else:
        pred_1.append(1)
        yval_1.append(yvals[i])
        
MSE_0 = mean_squared_error(pred_0, yval_0)
print("The MSE for category 0 is", MSE_0)

MSE_1 = mean_squared_error(pred_1, yval_1)
print("The MSE for category 1 is", MSE_1)

The MSE for the RF is 0.0714285714286
The MSE for category 0 is 0.0578947368421
The MSE for category 1 is 0.0841584158416


#### (c) SVM

In [8]:
k = 4
kf = KFold(n_splits=k, random_state=15, shuffle=True)
kf.get_n_splits(Xvars)
err0_rate_vec = np.zeros(k)
err1_rate_vec = np.zeros(k)
MSE_vec_svm = np.zeros(k)

k_ind = int(0)
for train_index, test_index in kf.split(Xvars):
    X_train, X_test = Xvars[train_index], Xvars[test_index]
    y_train, y_test = yvals[train_index], yvals[test_index]
    
    svc = svm.SVC(kernel='rbf', gamma = 0.2, C=1)
    svc.fit(X_train, y_train)
    y_pred = svc.predict(X_test)
    y_test = y_test.flatten()
    
    #logical filter
    a = (y_test != y_pred)
    b0 = y_pred == 0
    b1 = y_pred == 1
    err0_rate = (b0 & a).sum() / b0.sum() 
    err1_rate = (b1 & a).sum() / b1.sum() 
    err0_rate_vec[k_ind] = err0_rate
    err1_rate_vec[k_ind] = err1_rate
    MSE_vec_svm[k_ind] = (y_test != y_pred).mean()
    
    print('When k =', k_ind)
    print(classification_report(y_test, y_pred))
    print('error rate for category 0 is', err0_rate)
    print('error rate for category 1 is', err1_rate)
    
    k_ind  += 1

print("============ summary ===============")
print('average MSE svm for both categories', MSE_vec_svm.mean())
print('average MSE standard err for both categories', MSE_vec_svm.std())
print('average MSE svm for category 0', err0_rate_vec.mean())
print('average MSE standard err for category 0', err0_rate_vec.std())
print('average MSE svm for category1', np.nanmean(err1_rate_vec))
print('average MSE standard err for category1', np.nanstd(err1_rate_vec))


When k = 0
             precision    recall  f1-score   support

          0       1.00      0.04      0.07        55
          1       0.45      1.00      0.62        43

avg / total       0.76      0.46      0.31        98

error rate for category 0 is 0.0
error rate for category 1 is 0.552083333333
When k = 1
             precision    recall  f1-score   support

          0       0.48      1.00      0.65        47
          1       0.00      0.00      0.00        51

avg / total       0.23      0.48      0.31        98

error rate for category 0 is 0.520408163265
error rate for category 1 is nan
When k = 2
             precision    recall  f1-score   support

          0       0.47      1.00      0.64        45
          1       1.00      0.04      0.07        53

avg / total       0.76      0.48      0.33        98

error rate for category 0 is 0.53125
error rate for category 1 is 0.0
When k = 3
             precision    recall  f1-score   support

          0       0.53      1.00 

C:\Users\Lenovo\Anaconda3\lib\site-packages\sklearn\utils\validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Lenovo\Anaconda3\lib\site-packages\ipykernel\__main__.py:23: RuntimeWarning: invalid value encountered in long_scalars
C:\Users\Lenovo\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


#### (d)

Purely judging by the MSE performance, Random Forest is the best, followed by Logit regression while SVM performs the worst among the three.